### Code Flow and Explanation:

1. **Import Libraries**:
   - Import `pandas` for data manipulation.

2. **Data Loading**:
   - Load interaction data (`Combined_Interactions.xlsx`), event details (`Events.xlsx`), and user responses (`(Responses) (1).csv`) into dataframes.

3. **Filter User Data**:
   - Filter interactions and responses for a specific user (e.g., `Pratik Bhande`).
   - Split user interests into a list for comparison.

4. **Matching Interests**:
   - Combine primary and secondary event categories into a list (`Category`) for each event.
   - Match user interests against the event categories. Mark events that match the user's interests.

5. **Merge Data**:
   - Merge the user's interaction data with event details to obtain both the event categories and interaction scores.
   - Separate out interactions with positive scores and those with negative or zero scores.

6. **Recommendations Logic**:
   - **Category-Based Recommendations**:
     - Iterate through event categories and recommend events within subcategories the user hasn't interacted with.
     - Ensure at least one event per category is recommended.
   - **Unrecommended Categories**:
     - For categories that haven't been recommended yet, select other subcategories for recommendations.
   - **Additional Events**:
     - Add more events based on the user's interaction score to ensure diversity in recommendations.

7. **Return Recommendations**:
   - Return the final recommended events after sorting.

8. **Example Usage**:
   - The function is called with the user `Pratik Bhande` to generate event recommendations, and the top 5 recommendations are displayed.

In [3]:
import pandas as pd

def recommend_events_for_user(user_name):
    # Read data
    interactions_df = pd.read_excel('Combined_Interactions.xlsx')
    events_df = pd.read_excel('Events.xlsx')
    responses = pd.read_csv('(Responses) (1).csv')

    # Filter interactions for the specific user
    interactions_df = interactions_df[interactions_df['Name'] == user_name]

    # Filter responses for the specific user and split interests
    responses = responses[responses['Name'] == user_name]
    responses['All Interests'] = responses['All Interests'].str.split(', ')

    # Matching interests
    events_df['Category'] = events_df[['Primary Category', 'Secondary Category']].apply(lambda x: ', '.join(filter(lambda y: pd.notna(y), x)), axis=1)
    events_df['Category'] = events_df['Category'].str.split(', ')
    events_df['Interest_Match'] = False

    for index, row in responses.iterrows():
        for tag in row['All Interests']:
            events_df.loc[events_df['Category'].apply(lambda x: tag in x), 'Interest_Match'] = True
    events_df = events_df[events_df['Interest_Match']==True]

    # Merge data
    filtered_interactions_df_1 = interactions_df[~interactions_df['Interaction Score'].isin([-1, 0])]
    merged_df = pd.merge(interactions_df, events_df[['Event ID', 'Primary Category', 'Sub Category']], on='Event ID', how='left', suffixes=('_int', '_evt'))
    merged_df_1 = pd.merge(filtered_interactions_df_1, events_df[['Event ID', 'Primary Category', 'Sub Category']], on='Event ID', how='left', suffixes=('_int', '_evt'))
    filtered_interactions_df_neg_zero = interactions_df[interactions_df['Interaction Score'].isin([-1, 0])]

    # Recommendations
    recommended_events = pd.DataFrame()
    categories = merged_df.sort_values('Interaction Score', ascending=False)['Primary Category_evt'].unique()
    recommended_categories = set()

    for category in categories:
        category_events = events_df[events_df['Primary Category'] == category]
        user_subcategories = merged_df_1[merged_df_1['Primary Category_evt'] == category]['Sub Category_evt'].unique()

        for subcategory in user_subcategories:
            subcategory_events = category_events[category_events['Sub Category'] == subcategory]
            new_events = subcategory_events[~subcategory_events['Event ID'].isin(merged_df_1['Event ID']) &
                                            ~subcategory_events['Event ID'].isin(recommended_events) &
                                            ~subcategory_events['Event ID'].isin(filtered_interactions_df_neg_zero['Event ID'])]

            if not new_events.empty:
                recommended_categories.add(category)
                recommended_events = pd.concat([recommended_events, new_events.head(1)], ignore_index=True)

    # Add unrecommended events
    unrecommended_categories = set(categories) - recommended_categories
    for ur_category in unrecommended_categories:
        category_events_1 = events_df[events_df['Primary Category'] == ur_category]
        user_subcategories_1 = merged_df[merged_df['Primary Category_evt'] == ur_category]['Sub Category_evt'].unique()

        for sc in user_subcategories_1:
            subcategory_events_1 = category_events_1[category_events_1['Sub Category'] != sc]
            ur_category_events = subcategory_events_1[
                (~subcategory_events_1['Event ID'].isin(merged_df['Event ID'])) &
                (~subcategory_events_1['Event ID'].isin(recommended_events['Event ID']))]

            recommended_events = pd.concat([recommended_events, ur_category_events.head(1)], ignore_index=True)

    # Add additional events based on sorting
    sorted_events = merged_df.sort_values('Interaction Score', ascending=False)
    for index, event in sorted_events.iterrows():
        additional_events = events_df[
            (events_df['Primary Category'] == event['Primary Category_evt']) &
            (~events_df['Event ID'].isin(merged_df['Event ID'])) &
            (~events_df['Event ID'].isin(recommended_events['Event ID']))]

        recommended_events = pd.concat([recommended_events, additional_events.head(1)], ignore_index=True)

    return recommended_events

# Call the function with the user name
recommended_events = recommend_events_for_user('Pratik Bhande')
recommended_events.head(5)

,Event ID,Event Names,Description,Primary Category,Secondary Category,Sub Category,Location,Prices,Date,Unnamed: 9,Time,Links,Category,Interest_Match
0,EVT0081,Kolad River Side Camping and Water Sports,*Dear people* Adventure Camp Zone ( Kolad )Get...,Adventure,Travel,Adventure Sports,Deccan,1500,2024-03-31 00:00:00,Sunday,15:00:00,https://allevents.in/roha/kolad-river-side-cam...,"[Adventure, Travel]",True
1,EVT0085,Dandeli Jungle Tour,उनाड भ्रमंती is happy to invite you for *Dande...,Adventure,Travel,Hiking and Trekking,Sangamvadi,10000,2024-03-29 00:00:00,Friday,00:00:00,https://allevents.in/pune/dandeli-jungle-tour-...,"[Adventure, Travel]",True
2,EVT0052,Friday Diva Night - DJ Veronica,CELEBRATE FRIDAY DIVA NIGHT WITH CHEERS TO YOU...,Music and Entertainment,NaN,Concert,Koregaon Park,2500,2024-11-18 00:00:00,Monday,18:00:00,https://allevents.in/pune/friday-diva-night-dj...,[Music and Entertainment],True
3,EVT0068,Theatre Shows,Join us for an enchanting Theatre Workshop des...,Music and Entertainment,NaN,Drama,Deccan,NaN,Multiple Dates,Multiple Dates,10:00:00,NaN,[Music and Entertainment],True
4,EVT0031,Pharma and Lab Expo,NaN,Tech and Innovation,NaN,Exhibition,Kharadi,400,2024-07-14 00:00:00,Sunday,17:00:00,NaN,[Tech and Innovation],True
